In [44]:
import pandas as pd
import numpy as np
import gcsfs
import recordlinkage as rl
import matching as m
import helpers as h
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [65]:
file_paths = gcs.ls('soleadify_sample_data/facebook_google_matches')
facebook_google_matches = pd.DataFrame()
for file_path in file_paths:
    if file_path[-4:] == '.csv':
        with gcs.open(file_path) as f:
            subset = pd.read_csv(f, quotechar='"', escapechar='\\')
        subset = subset.reindex(sorted(subset.columns), axis=1)
        print(subset.shape)
        facebook_google_matches = pd.concat([facebook_google_matches, subset])

(7, 30)
(310, 30)
(2, 30)
(2322, 30)
(1307, 30)


In [66]:
facebook_google_matches.shape

(3948, 30)

In [67]:
file_paths = gcs.ls('soleadify_sample_data/google_website_matches')
google_website_matches = pd.DataFrame()
for file_path in file_paths:
    if file_path[-4:] == '.csv':
        with gcs.open(file_path) as f:
            subset = pd.read_csv(f, quotechar='"', escapechar='\\')
        subset = subset.reindex(sorted(subset.columns), axis=1)
        print(subset.shape)
        google_website_matches = pd.concat([google_website_matches, subset])

(1, 28)
(158, 28)
(3, 28)
(352, 28)
(475, 28)
(7, 28)


In [68]:
google_website_matches.shape

(996, 28)

In [12]:
facebook_google_matches.columns

Index(['address__facebook', 'address__google', 'categories', 'category',
       'city_clean__facebook', 'city_clean__google', 'country_clean__facebook',
       'country_clean__google', 'description', 'email', 'index__facebook',
       'index__google', 'link', 'page_type', 'phone__facebook',
       'phone__google', 'raw_address', 'raw_phone', 'region_clean__facebook',
       'region_clean__google', 'region_code__facebook', 'region_code__google',
       'region_name', 'root_domain__facebook', 'root_domain__google',
       'site_name_clean__facebook', 'site_name_clean__google', 'text',
       'zip_code__facebook', 'zip_code__google'],
      dtype='object')

In [69]:
renamers = {'categories':'category__facebook'
            ,'category':'category__google'
           ,'description':'description__facebook'
            ,'text':'description__google'
           ,'email':'email__facebook'
           ,'link':'link__facebook'
           ,'page_type':'page_type__facebook'
           ,'raw_address':'raw_address__google'
           ,'raw_phone':'raw_phone__google'
           }
to_drop = ['region_name','region_code__facebook','region_code__google']
facebook_google_matches = (facebook_google_matches
                        .drop(to_drop, axis=1)
                        .rename(columns = renamers)
                          )

In [13]:
google_website_matches.columns

Index(['address', 'category', 'category_clean', 'city_clean__google',
       'city_clean__website', 'country_clean__google',
       'country_clean__website', 'domain_suffix', 'index__google',
       'index__website', 'language', 'legal_name_clean', 'phone__google',
       'phone__website', 'raw_address', 'raw_phone', 'region_clean__google',
       'region_clean__website', 'region_code', 'region_name',
       'root_domain__google', 'root_domain__website', 'site_name',
       'site_name_clean__google', 'site_name_clean__website', 'text', 'tld',
       'zip_code'],
      dtype='object')

In [70]:
renamers = {'address':'address__google'
            ,'category':'category__google'
            ,'category_clean':'category__website'
            ,'language':'language__website'
            ,'legal_name_clean':'legal_name__website'
            ,'text':'description__google'
           ,'raw_address':'raw_address__google'
           ,'raw_phone':'raw_phone__google'
            ,'zip_code':'zip_code__google'
           }
to_drop = ['domain_suffix','tld','region_code','region_name','site_name']
google_website_matches = (google_website_matches
                        .drop(to_drop, axis=1)
                        .rename(columns = renamers)
                          )

In [71]:
cols_1 = list(google_website_matches.columns)
cols_1.sort()

In [72]:
cols_2 = list(facebook_google_matches.columns)
cols_2.sort()

In [73]:
cols_1_df = pd.Series(cols_1).rename('google_website_matches').to_frame()
cols_2_df = pd.Series(cols_2).rename('facebook_google_matches').to_frame()
columns = cols_1_df.merge(cols_2_df, how='outer', left_on='google_website_matches'
                          , right_on='facebook_google_matches')

In [47]:
columns

,google_website_matches,facebook_google_matches
0,address__google,address__google
1,category__google,category__google
2,category__website,NaN
3,city_clean__google,city_clean__google
4,city_clean__website,NaN
5,country_clean__google,country_clean__google
6,country_clean__website,NaN
7,description__google,description__google
8,index__google,index__google
9,index__website,NaN


In [74]:
for _,row in columns.iterrows():
    if pd.isna(row['google_website_matches']):
        google_website_matches[row['facebook_google_matches']] = np.nan
    elif pd.isna(row['facebook_google_matches']):
        facebook_google_matches[row['google_website_matches']] = np.nan

In [75]:
google_website_matches = google_website_matches.reindex(sorted(google_website_matches.columns), axis=1)
facebook_google_matches = facebook_google_matches.reindex(sorted(facebook_google_matches.columns), axis=1)
matches_1 = pd.concat([google_website_matches, facebook_google_matches])

In [76]:
matches_1.shape

(4944, 37)

In [77]:
with gcs.open('soleadify_sample_data/unified_matched_data/matches_1.csv','w') as f:
    matches_1.to_csv(f,index = False, quotechar='"', escapechar='\\')

In [ ]:
file_paths = gcs.ls('soleadify_sample_data/facebook_website_matches')
facebook_website_matches = pd.DataFrame()
for index, file_path in enumerate(file_paths):
    if file_path[-4:] == '.csv':
        print(index)
        with gcs.open(file_path) as f:
            if file_path == 'soleadify_sample_data/facebook_website_matches/subset_of_matches_188.csv':
                subset = pd.read_csv(f, quotechar='"', escapechar='\\', skiprows = [29])
            else:
                subset = pd.read_csv(f, quotechar='"', escapechar='\\')
        subset = subset.reindex(sorted(subset.columns), axis=1)
        facebook_website_matches = pd.concat([facebook_website_matches, subset])

In [92]:
facebook_website_matches.columns

Index(['address', 'categories', 'category_clean', 'city_clean__facebook',
       'city_clean__website', 'country_clean__facebook',
       'country_clean__website', 'description', 'domain_suffix', 'email',
       'index__facebook', 'index__website', 'language', 'legal_name_clean',
       'link', 'page_type', 'phone__facebook', 'phone__website',
       'region_clean__facebook', 'region_clean__website', 'region_code',
       'root_domain__facebook', 'root_domain__website', 'site_name',
       'site_name_clean__facebook', 'site_name_clean__website', 'tld',
       'zip_code'],
      dtype='object')

In [99]:
renamers = {'categories':'category__facebook'
            ,'address':'address__facebook'
            ,'category_clean':'category__website'
            ,'zip_code':'zip_code__facebook'
           ,'description':'description__facebook'
           ,'email':'email__facebook'
           ,'link':'link__facebook'
           ,'page_type':'page_type__facebook'
            ,'legal_name_clean':'legal_name__website'
            ,'language':'language__website'
           }
to_drop = ['domain_suffix','tld','region_code','site_name']
facebook_website_matches = (facebook_website_matches
                        .drop(to_drop, axis=1)
                        .rename(columns = renamers)
                          )

In [100]:
matches_1.shape

(4944, 37)

In [101]:
cols_1 = list(matches_1)
cols_1.sort()

cols_2 = list(facebook_website_matches)
cols_2.sort()

cols_1_df = pd.Series(cols_1).rename('matches_1').to_frame()
cols_2_df = pd.Series(cols_2).rename('facebook_website_matches').to_frame()
columns = cols_1_df.merge(cols_2_df, how='outer', left_on='matches_1'
                          , right_on='facebook_website_matches')

In [102]:
columns

,matches_1,facebook_website_matches
0,address__facebook,address__facebook
1,address__google,NaN
2,category__facebook,category__facebook
3,category__google,NaN
4,category__website,category__website
5,city_clean__facebook,city_clean__facebook
6,city_clean__google,NaN
7,city_clean__website,city_clean__website
8,country_clean__facebook,country_clean__facebook
9,country_clean__google,NaN


In [103]:
for _,row in columns.iterrows():
    if pd.isna(row['matches_1']):
        matches_1[row['facebook_website_matches']] = np.nan
    elif pd.isna(row['facebook_website_matches']):
        facebook_website_matches[row['matches_1']] = np.nan

In [104]:
matches_1 = matches_1.reindex(sorted(matches_1.columns), axis=1)
facebook_website_matches = facebook_website_matches.reindex(sorted(facebook_website_matches.columns), axis=1)
matches_full = pd.concat([matches_1, facebook_website_matches])

In [105]:
matches_full.shape

(29761, 37)

In [106]:
with gcs.open('soleadify_sample_data/unified_matched_data/matches_full.csv','w') as f:
    matches_full.to_csv(f,index = False, quotechar='"', escapechar='\\')